In [1]:
import pandas as pd
import numpy as np
from glob import glob
from ts_helper_functions1 import *

# Brief tutorial on use of ts_helpers functions

### Import a dataset to test with

In [3]:
df = pd.read_excel(files[0], sheet_name= 'Daily', parse_dates= ['Date'])
df.head()

,Date,Day,Orders,CST Tickets,All Tickets,Unique Requesters,Average Time/CST Ticket,Revenue,Tax,Shipping,Net,Week,Orders/Week,CST Tickets/Week,All Tickets/Week,Unique Requesters/Week,Avg Time/Ticket/Week,Net/Week
0,2016-01-01,NaN,251.0,12.0,42.0,8.0,2398.333333,23400.05,0.0,734.60,22665.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-01-02,NaN,288.0,15.0,52.0,12.0,508.733333,23513.11,0.0,1092.10,22421.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-01-03,NaN,327.0,35.0,62.0,18.0,241.485714,16849.05,0.0,905.65,15943.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-01-04,1.0,845.0,47.0,299.0,96.0,435.659574,59523.53,0.0,2210.35,57313.18,1.0,4215.0,564.0,1683.0,535.0,274.882979,384785.07
4,2016-01-05,2.0,707.0,150.0,260.0,78.0,288.700000,78214.64,0.0,2053.60,76161.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# help() function will show the doc string
help(create_splits)

Help on function create_splits in module ts_helper_functions1:

create_splits(df: pandas.core.frame.DataFrame, date_col: str, target: str, num_validations: int, training_duration: int, validation_duration: int, holdout: bool = True, gap: int = 0, series_id=False, specific_dates=False, split_pct=False, trim_leading_zeros: bool = True, trim_lagging_zeros: bool = True) -> dict
    A function to create backtests for partitioning time_series modeling.
    
    Inputs:
    ------
    df: Pandas dataframe with training data
    date_col: Name of primary-datetime column in dataframe
    target: Name of target-column in dataframe
    num_validations: Number of backtests to create (excluding holdout)
    training_duration: Number of rows for the training data
    validation_duration: Number of rows for the validation data
    holdout: Bool, generate holdout with most recent data
    gap: Number of rows between end of training data and start of validation
    specific dates: Bool, or a list of tu

## First use case: Duration-based
- Define the duration of training and validation periods
- Define the number of validations (i.e. backtests)
- Define if a holdout is used, holdout will always be the most recent data
- If you use a holdout, you will be (num_validations + 1) total partitions
- You can choose to remove any leading/lagging null- or zero- target rows

In [5]:
df_splits = create_splits(df= df,
             date_col= 'Date',
             target= 'CST Tickets',
             num_validations= 1,
             holdout= True,
             training_duration= 365*3,
             validation_duration= 90,
             gap= 0,
             specific_dates = False,
             split_pct= False,
             trim_leading_zeros= True,
             trim_lagging_zeros= True)

Creating Backtests: 100%|██████████| 2/2 [00:00<00:00, 117.37it/s]

Leading bad rows removed:  1
Lagging bad rows removed:  244
*** Holdout Created ***
*** Backtest1 Created ***


In [6]:
# the output of the function is a dict
df_splits.keys()

dict_keys(['stats', 'training', 'validation'])

In [7]:
# stats contains basic dataset stats on the original dataframe
df_splits['stats']

{'date_col': 'Date',
 'target': 'CST Tickets',
 'series_id': False,
 'rows': 1950,
 'columns': 18,
 'min_target': 1.0,
 'max_target': 8534.0,
 'start_date': Timestamp('2016-01-02 00:00:00'),
 'end_date': Timestamp('2021-05-04 00:00:00'),
 'timespan': Timedelta('1949 days 00:00:00'),
 'median_timestep': Timedelta('1 days 00:00:00'),
 'min_timestep': Timedelta('1 days 00:00:00'),
 'max_timestep': Timedelta('1 days 00:00:00'),
 'time_step': 1.0,
 'series_time_steps': nan}

In [8]:
# the 'training' key contains the datasets for training
df_splits['training'].keys()

dict_keys(['holdout', 'backtest1'])

In [9]:
backtest1_training = df_splits['training']['backtest1']
backtest1_training.head(3)

,Date,Day,Orders,CST Tickets,All Tickets,Unique Requesters,Average Time/CST Ticket,Revenue,Tax,Shipping,Net,Week,Orders/Week,CST Tickets/Week,All Tickets/Week,Unique Requesters/Week,Avg Time/Ticket/Week,Net/Week
673,2017-11-04,307.0,1507.0,211.0,417.0,111.0,273.600000,158791.93,0.0,4666.60,154125.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN
674,2017-11-05,308.0,2306.0,145.0,358.0,96.0,288.819444,107630.17,0.0,4321.85,103308.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN
675,2017-11-06,309.0,4348.0,346.0,1792.0,307.0,485.027523,313053.90,0.0,9307.90,303746.00,45.0,22002.0,2064.0,7659.0,2550.0,524.066629,1739368.46


In [10]:
# the 'validation' key contains the matched datasets for validation
df_splits['validation'].keys()

dict_keys(['holdout', 'backtest1'])

In [11]:
backtest1_validation = df_splits['validation']['backtest1']
backtest1_validation.head(3)

,Date,Day,Orders,CST Tickets,All Tickets,Unique Requesters,Average Time/CST Ticket,Revenue,Tax,Shipping,Net,Week,Orders/Week,CST Tickets/Week,All Tickets/Week,Unique Requesters/Week,Avg Time/Ticket/Week,Net/Week
1769,2020-11-04,311.0,7319.0,1365.0,3159.0,1235.0,253.631032,672933.36,37806.07,24879.53,610247.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1770,2020-11-05,312.0,7206.0,1364.0,3151.0,1110.0,280.312453,670356.42,37210.94,24001.72,609143.76,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1771,2020-11-06,313.0,6610.0,1205.0,2760.0,984.0,250.047904,617031.50,35037.12,22619.52,559374.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
print('holdout training start: ', df_splits['training']['holdout']['Date'].min())
print('holdout training end: ', df_splits['training']['holdout']['Date'].max())
print('holdout validation start: ', df_splits['validation']['holdout']['Date'].min())
print('holdout validation end: ', df_splits['validation']['holdout']['Date'].max(), "\n")

print('backtest1 training start: ', df_splits['training']['backtest1']['Date'].min())
print('backtest1 training end: ', df_splits['training']['backtest1']['Date'].max())
print('backtest1 validation start: ', df_splits['validation']['backtest1']['Date'].min())
print('backtest1 validation end: ', df_splits['validation']['backtest1']['Date'].max())

holdout training start:  2018-02-03 00:00:00
holdout training end:  2021-02-02 00:00:00
holdout validation start:  2021-02-03 00:00:00
holdout validation end:  2021-05-04 00:00:00 

backtest1 training start:  2017-11-04 00:00:00
backtest1 training end:  2020-11-03 00:00:00
backtest1 validation start:  2020-11-04 00:00:00
backtest1 validation end:  2021-02-02 00:00:00


## Second use case: Explicitly defined dates
- Define the start and end for training and validation as a list of tuples ex. [(tuple1), (tuple2)]
- Each tuple corresponds to holdout or backtest partition: (training-start, training-end, validation-start, validation-end)
- Define the number of total partitions (holdout + backtests) (needs to match with length of list)
- Define if a holdout is used, holdout will always be the first entry in the list of tuples

In [13]:
# dates can be defined as strings or datetime type objects
date_splits = [('2018-02-03', '2021-02-02', '2021-02-03', '2021-05-04'),
              ('2017-11-04', '2020-11-03', '2020-11-04', '2021-02-02')]

In [14]:
df_splits = create_splits(df= df,
             date_col= 'Date',
             target= 'CST Tickets',
             num_validations= 1,
             holdout= True,
             training_duration= 365*3,
             validation_duration= 90,
             gap= 0,
             specific_dates = date_splits,
             split_pct= False)

Creating Backtests: 100%|██████████| 2/2 [00:00<00:00, 134.69it/s]

Leading bad rows removed:  1
Lagging bad rows removed:  244
*** Holdout Created ***
*** Backtest1 Created ***


In [15]:
print('holdout training start: ', df_splits['training']['holdout']['Date'].min())
print('holdout training end: ', df_splits['training']['holdout']['Date'].max())
print('holdout validation start: ', df_splits['validation']['holdout']['Date'].min())
print('holdout validation end: ', df_splits['validation']['holdout']['Date'].max(), "\n")

print('backtest1 training start: ', df_splits['training']['backtest1']['Date'].min())
print('backtest1 training end: ', df_splits['training']['backtest1']['Date'].max())
print('backtest1 validation start: ', df_splits['validation']['backtest1']['Date'].min())
print('backtest1 validation end: ', df_splits['validation']['backtest1']['Date'].max())

holdout training start:  2018-02-03 00:00:00
holdout training end:  2021-02-02 00:00:00
holdout validation start:  2021-02-03 00:00:00
holdout validation end:  2021-05-04 00:00:00 

backtest1 training start:  2017-11-04 00:00:00
backtest1 training end:  2020-11-03 00:00:00
backtest1 validation start:  2020-11-04 00:00:00
backtest1 validation end:  2021-02-02 00:00:00


## Third use: Define a Percent for the split of training/validation
- Provide a float that represents what percent of the data should be used for training 
- Define the number of backbests (use only 1 for split_pct)
- Define if a holdout is used (don't use this for split_pct)

#### Note: This use case is explicitly designed for the use of internal-splits for hyperparameter tuning. - 
- In brief, you actually want to split your training data (80/20 is a good default) and perform your hyperparameter tuning validation on the last 20% of training data. 
- You then select the best model/approach hyperparameters based on that training-split, and then test that on the actual validation that corresponds to the training. 
- It ensures that the validation is actually out-of-sample, and a truly fair comparison to performance on other validation folds using the same hyperparameters.
- As a result, this split_pct is only desgined to generate a single validation/split

In [16]:
df_splits = create_splits(df= df,
             date_col= 'Date',
             target= 'CST Tickets',
             num_validations= 1,
             holdout= False,
             training_duration= 365*3,
             validation_duration= 90,
             gap= 0,
             specific_dates = False,
             split_pct= 0.8)

Creating Backtests: 100%|██████████| 1/1 [00:00<00:00, 221.45it/s]

Leading bad rows removed:  1
Lagging bad rows removed:  244
*** Backtest1 Created ***


In [18]:
print('backtest1 training start: ', df_splits['training']['backtest1']['Date'].min())
print('backtest1 training end: ', df_splits['training']['backtest1']['Date'].max())
print('backtest1 validation start: ', df_splits['validation']['backtest1']['Date'].min())
print('backtest1 validation end: ', df_splits['validation']['backtest1']['Date'].max())

backtest1 training start:  2016-01-02 00:00:00
backtest1 training end:  2020-04-09 00:00:00
backtest1 validation start:  2020-04-10 00:00:00
backtest1 validation end:  2021-05-04 00:00:00
